In [6]:
from torch import nn

In [27]:
def create_sequential(num_layers, 
                      nodes_per_layer, 
                      activation=nn.ReLU, 
                      final_activation = None):
    """
    Create a Sequential module with the specified number of layers and nodes.

    Args:
        num_layers (int): Number of layers in the Sequential module.
        nodes_per_layer (list[int]): List of integers specifying the number of nodes in each layer.
                                     Must have `num_layers + 1` elements (input size + output sizes).
        activation (nn.Module): The activation function to use between layers. Default is nn.ReLU.
        final_activation (nn.Module): Optional final activation function after the last layer.

    Returns:
        nn.Sequential: The constructed Sequential module.
    """
    if len(nodes_per_layer) != num_layers + 1:
        raise ValueError('Number of nodes per layer must have the same length as the number of layers + 1!')
    layers = []
    for i in range(num_layers):
        layers.append(nn.Linear(nodes_per_layer[i], nodes_per_layer[i+1]))
        if i < num_layers-1:
            layers.append(activation())
    if final_activation is not None:
        layers.append(final_activation())
    
    return nn.Sequential(*layers)



def replace_final_layer(
        model, 
        num_layers=4, 
        nodes_per_layer=[128,64,32,1],
        activation='nn.ReLU',
        final_activation='None',
    ):
    """
    Replace the final layer of a PyTorch model with a new layer.

    Args:
        model (nn.Module): A PyTorch model instance.
        new_layer (nn.Module): The new layer to replace the final layer with.
    
    Returns:
        nn.Module: The model with the final layer replaced.
    """
    
    activation = eval(activation)
    final_activation = eval(final_activation)
    # print(f"[DEBUG] Activation {activation} and final activation {final_activation}")
    children = list(model.named_children())
    # print(f"[DEBUG] Children: {len(children)} of class {type(children)}")
    last_name, last_module = children[-1]
    new_layer_in = last_module.in_features
    
    # last_module_out = last_module._out
    new_layer = create_sequential(
        num_layers, 
        [new_layer_in] + nodes_per_layer,
        activation,
        final_activation)
    # print(f"[DEBUG] Last module {last_module} and new_layer {new_layer}")
    setattr(model, last_name, new_layer)
    
    return model

In [28]:
from torchvision import models

model = models.resnet18()

In [29]:
model = replace_final_layer(model=model)

In [30]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [1]:
initializer_configs = {
    'loss_fn': 'torch.nn.MSELoss',
    'optimizer': {
        'optimizer': 'torch.optim.Adam',
        'amsgrad': 'True'},
    'scheduler': {'scheduler': 'torch.optim.lr_scheduler.StepLR',
                  'step_size': 7,
                  'gamma': 0.1},
    'metric': 'torcheval.metrics.R2Score'  
}

In [4]:
initializer_configs = {
    'loss_fn': 'torch.nn.MSELoss',
    'loss_fn_params': None,
    'optimizer': 'torch.optim.Adam',
    'optimizer_params': {
        'amsgrad': 'True'
        },
    'scheduler': 'torch.optim.lr_scheduler.StepLR',
    'scheduler_params':{
        'step_size': 7,
        'gamma': 0.1
    },
    'metric': 'torcheval.metrics.R2Score',
    'metric_params': None
}

In [5]:
initializer_configs

{'loss_fn': 'torch.nn.MSELoss',
 'loss_fn_params': None,
 'optimizer': 'torch.optim.Adam',
 'optimizer_params': {'amsgrad': 'True'},
 'scheduler': 'torch.optim.lr_scheduler.StepLR',
 'scheduler_params': {'step_size': 7, 'gamma': 0.1},
 'metric': 'torcheval.metrics.R2Score',
 'metric_params': None}

In [6]:
import importlib

def string_to_callable(callable_string):
    """
    Convert a string to a callable. This function helps to convert Kedro's
    parameters into callables that can be passed to model building functions
    in pipeline.
    """
    if callable_string == 'None':
        return None
    module_name, function_name = callable_string.rsplit('.', 1)

    print(f'[INFO] Module called {module_name} with a function name: {function_name}')

    module = importlib.import_module(module_name)
    func = getattr(module, function_name)
    return func

In [7]:
def instantiate_class_with_params(
        class_name='torch.optim.lr_scheduler.StepLR', 
        parameters={'step_size': 7, 'gamma': 0.1}):
    return string_to_callable(class_name)(**parameters)

In [8]:
instantiate_class_with_params()

[INFO] Module called torch.optim.lr_scheduler with a function name: StepLR


TypeError: StepLR.__init__() missing 1 required positional argument: 'optimizer'

In [2]:
my_dict = {'a': 1, 'b': 2}
my_dict['ananas'] = 1

In [3]:
my_dict

{'a': 1, 'b': 2, 'ananas': 1}

In [4]:
my_dict.items()

dict_items([('a', 1), ('b', 2), ('ananas', 1)])